In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision torchtext spacy
import torch

In [2]:
torch.cuda.is_available()

True

In [3]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
!ls


sample_data


In [0]:
ls /gdrive/My\ Drive/ML/russian_wiki_text

HW_mod.ipynb  russian_wiki_text/  utils.py


In [0]:
!pwd

/content


In [0]:
import os
import torch
from random import random as rand

def read_file(f_name):
    contents = []
    texts = []
    with open(f_name) as f:
        for line in f:
            if line[0] == "<":
                if (line.startswith("<doc")):
                    contents = []
                    continue
                if (line.startswith("</doc>")):
                    s = "".join(contents)
                    texts.append(s)
                    continue
            contents.append(line)
    return texts

class Alphabet(object):
    def __init__(self):
        self.symbol2idx = {}
        self.idx2symbol = []
        self._len = 0
        
    def add_symbol(self, s):
        if s not in self.symbol2idx:
            self.idx2symbol.append(s)
            self.symbol2idx[s] = self._len
            self._len += 1
    
    def __len__(self):
        return self._len
    
import re    

class Texts(object):
    def __init__(self, path):
        self.db_path = path
        self.dictionary = Alphabet()
        self.train_files = self.load_files_list(os.path.join(path, 'train.txt'))
        self.valid_files = self.load_files_list(os.path.join(path, 'validation.txt'))
        self.test_files = self.load_files_list(os.path.join(path, 'test.txt'))
        print("Loading train...")
        self.train = self.tokenize(self.train_files, 0.2)
        print("Loading valid...")
        self.valid = self.tokenize(self.valid_files, 0.2)
        print("Loading test...")
        self.test = self.tokenize(self.test_files, 0.2)
        
        
    def load_files_list(self, path):
        lst = {}
        f = open(path, "r")
        for l in f.readlines():
            s = l.split()
            n = os.path.join(self.db_path, s[0])
            if n not in lst:
                lst[n] = []
            i = int(s[1])
            lst[n].append(i)
        return lst

    def tokenize(self, files_lst, prob):
        """Tokenizes a text file."""
        strings = []
        file_idx = 0
        regexp = re.compile("[^\ \,\.абвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ]")
        for f_name in files_lst.keys():
            file_idx += 1
            if rand() > prob:
                continue
            if file_idx%10 == 0:
                print ("Processing " + str(file_idx) + " of " + str(len(files_lst)))
            contents = read_file(f_name)
            for idx in files_lst[f_name]:
                s = contents[idx]
                s = regexp.sub('', s)
                strings.append(s)
        tokens = 0
        for line in strings:
            tokens += len(line)
            for s in line:
                self.dictionary.add_symbol(s)
        ids = torch.LongTensor(tokens)
        token = 0
        rec_idx = 0
        for line in strings:
            rec_idx += 1
            if rec_idx %10 == 0:
                print("Coding " + str(rec_idx) + " of " + str(len(strings)))
            for s in line:
                ids[token] = self.dictionary.symbol2idx[s]
                token += 1
        return ids

class TextLoader(object):
    def __init__(self, dataset, batch_size=128, sequence_length=30):
        self.data = dataset
        self.batch_size = batch_size
        self.sequence_length = sequence_length
        self._batchify()
        
    def _batchify(self):
        # Work out how cleanly we can divide the dataset into batch_size parts.
        self.nbatch = self.data.size(0) // self.batch_size
        # Trim off any extra elements that wouldn't cleanly fit (remainders).
        data = self.data.narrow(0, 0, self.nbatch * self.batch_size)
        # Evenly divide the data across the batch_size batches.
        self.batch_data = data.view(self.batch_size, -1).t().contiguous()
    
    def _get_batch(self, i):
        seq_len = min(self.sequence_length, len(self.batch_data) - 1 - i)
        data = self.batch_data[i:i+seq_len]
        target = self.batch_data[i+1:i+1+seq_len].view(-1)
        return data, target
    
    def __iter__(self):
        for i in range(0, self.batch_data.size(0) - 1, self.sequence_length):
            data, targets = self._get_batch(i)
            yield data, targets
    
    def __len__(self):
        return self.batch_data.size(0)



In [0]:
ls .

russian_wiki_text/  sample_data/


In [0]:
import os

import itertools
import pickle
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math 

import sys
sys.path.append('../')

%matplotlib inline

In [6]:
corpus = Texts("/gdrive/My Drive/ML/russian_wiki_text/")

Loading train...
Processing 10 of 5363
Processing 30 of 5363
Processing 50 of 5363
Processing 180 of 5363
Processing 230 of 5363
Processing 260 of 5363
Processing 340 of 5363
Processing 350 of 5363
Processing 460 of 5363
Processing 530 of 5363
Processing 580 of 5363
Processing 590 of 5363
Processing 660 of 5363
Processing 690 of 5363
Processing 770 of 5363
Processing 810 of 5363
Processing 870 of 5363
Processing 910 of 5363
Processing 940 of 5363
Processing 990 of 5363
Processing 1020 of 5363
Processing 1070 of 5363
Processing 1080 of 5363
Processing 1100 of 5363
Processing 1160 of 5363
Processing 1200 of 5363
Processing 1250 of 5363
Processing 1290 of 5363
Processing 1300 of 5363
Processing 1340 of 5363
Processing 1410 of 5363
Processing 1430 of 5363
Processing 1500 of 5363
Processing 1630 of 5363
Processing 1730 of 5363
Processing 1740 of 5363
Processing 1810 of 5363
Processing 1830 of 5363
Processing 1990 of 5363
Processing 2070 of 5363
Processing 2120 of 5363
Processing 2180 of 536

In [0]:
batch_size = 128
sequence_length = 30
grad_clip = 0.1
lr = 4.
best_val_loss = None
log_interval = 100

In [0]:
device = "cuda"

In [0]:
eval_batch_size = 128
train_loader = TextLoader(corpus.train, batch_size=batch_size)
val_loader = TextLoader(corpus.valid, batch_size=eval_batch_size)
test_loader = TextLoader(corpus.test, batch_size=eval_batch_size)

In [0]:
class RNNModel(nn.Module):

    def __init__(self, rnn_type, ntoken, ninp, nhid, nlayers, dropout=0.5):
        super(RNNModel, self).__init__()
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        if rnn_type == 'LSTM':
            self.rnn = nn.LSTM(ninp, nhid, nlayers, dropout=dropout)
        elif rnn_type == 'GRU':
            self.rnn = nn.GRU(ninp, nhid, nlayers, dropout=dropout)
        self.decoder = nn.Linear(nhid, ntoken)

        self.init_weights()

        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.fill_(0)
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, x, hidden=None):
        emb = self.drop(self.encoder(x))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded.view(output.size(0), output.size(1), decoded.size(1)), hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters()).data
        if self.rnn_type == 'LSTM':
            return (weight.new(self.nlayers, bsz, self.nhid).zero_(),
                    weight.new(self.nlayers, bsz, self.nhid).zero_())
        else:
            return weight.new(self.nlayers, bsz, self.nhid).zero_()

In [0]:
def evaluate(data_loader):
    model.eval()
    total_loss = 0
    ntokens = len(corpus.dictionary)
    hidden = model.init_hidden(eval_batch_size)
    for i, (data, targets) in enumerate(data_loader):
        data = data.to(device)
        targets = targets.to(device)
        output, hidden = model(data)
        output_flat = output.view(-1, ntokens)
        total_loss += len(data) * criterion(output_flat, targets).item()
    return total_loss / len(data_loader)

In [0]:
def train():
    model.train()
    total_loss = 0
    ntokens = len(corpus.dictionary)
    for batch, (data, targets) in enumerate(train_loader):
        data = data.to(device)
        targets = targets.to(device)
        model.zero_grad()
        output, hidden = model(data)
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
        for p in model.parameters():
            p.data.add_(-lr, p.grad.data)

        total_loss += loss.item()

        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_loader) // sequence_length, lr, cur_loss, math.exp(cur_loss)))
            total_loss = 0

In [0]:
ntokens = len(corpus.dictionary)
model = RNNModel('LSTM', ntokens, 128, 200, 3, 0.3).to(device)
criterion = nn.CrossEntropyLoss()

In [0]:
def generate(n=50, temp=1.):
    model.eval()
    x = torch.rand(1, 1).mul(ntokens).long().to(device)
    hidden = None
    out = []
    for i in range(n):
        output, hidden = model(x, hidden)
        s_weights = output.squeeze().data.div(temp).exp()
        s_idx = torch.multinomial(s_weights, 1)[0]
        x.data.fill_(s_idx)
        s = corpus.dictionary.idx2symbol[s_idx]
        out.append(s)
    return ''.join(out)

In [15]:
with torch.no_grad():
    print('sample:\n', generate(50), '\n')

for epoch in range(1, 206):
    train()
    val_loss = evaluate(val_loader)
    print('-' * 89)
    print('| end of epoch {:3d} | valid loss {:5.2f} | valid ppl {:8.2f}'.format(
        epoch, val_loss, math.exp(val_loss)))
    print('-' * 89)
    if not best_val_loss or val_loss < best_val_loss:
        best_val_loss = val_loss
    else:
        # Anneal the learning rate if no improvement has been seen in the validation dataset.
        lr /= 4.0
    with torch.no_grad():
        print('sample:\n', generate(50), '\n')


sample:
 .МХыА лшчгБь.бЭЗБЕьЩчЭЙыНЁЯкУ КмъЛкгЬбЮИфДРЗнЗшяГы 

| epoch   1 |   100/ 9837 batches | lr 4.00 | loss  3.35 | ppl    28.62
| epoch   1 |   200/ 9837 batches | lr 4.00 | loss  3.24 | ppl    25.64
| epoch   1 |   300/ 9837 batches | lr 4.00 | loss  3.23 | ppl    25.27
| epoch   1 |   400/ 9837 batches | lr 4.00 | loss  3.23 | ppl    25.27
| epoch   1 |   500/ 9837 batches | lr 4.00 | loss  3.23 | ppl    25.38
| epoch   1 |   600/ 9837 batches | lr 4.00 | loss  3.23 | ppl    25.28
| epoch   1 |   700/ 9837 batches | lr 4.00 | loss  3.23 | ppl    25.37
| epoch   1 |   800/ 9837 batches | lr 4.00 | loss  3.20 | ppl    24.54
| epoch   1 |   900/ 9837 batches | lr 4.00 | loss  3.05 | ppl    21.20
| epoch   1 |  1000/ 9837 batches | lr 4.00 | loss  2.88 | ppl    17.89
| epoch   1 |  1100/ 9837 batches | lr 4.00 | loss  2.83 | ppl    16.89
| epoch   1 |  1200/ 9837 batches | lr 4.00 | loss  2.79 | ppl    16.34
| epoch   1 |  1300/ 9837 batches | lr 4.00 | loss  2.78 | ppl    16.12
| 

KeyboardInterrupt: ignored